In [13]:

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr # oh yeah!

# imports

from IPython.display import Markdown, display

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [5]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyAI


In [6]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

In [7]:
# A generic system message - no more snarky adversarial AIs!

system_message = "You are a helpful assistant"

In [8]:

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [9]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [10]:
view = gr.Interface(
    fn=stream_claude,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


In [ ]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [14]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a list of houses of this website in markdown. \
Include the price of the house, and the living area \n\n"
    user_prompt += website.text
    return user_prompt

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

web = Website("https://crossroadsamsterdam.nl/woningaanbod/verdieping-26/")
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a table of houseprices, in a markdown file"
# user_prompt = user_prompt_for(web)

display_summary("https://crossroadsamsterdam.nl/woningaanbod/verdieping-26/")



Here is a summary of the available properties from the website "Crossroads Amsterdam | Verdieping 26", including their prices and living areas in markdown format:

```markdown
| Address          | Price (€) | Living Area (m²) | Number of Bedrooms | Status     |
|------------------|-----------|-------------------|--------------------|------------|
| Radarweg 297E    | 1,588     | 68.2              | 2                  | verhuurd   |
| Radarweg 297G    | 1,602     | 66.4              | 2                  | verhuurd   |
| Radarweg 297C    | 1,589     | 65.2              | 2                  | verhuurd   |
| Radarweg 297B    | 1,589     | 65.2              | 2                  | verhuurd   |
| Radarweg 297A    | 1,636     | 68.2              | 2                  | verhuurd   |
| Radarweg 297F    | 1,471     | 65.4              | 2                  | verhuurd   |
| Radarweg 297E    | 1,556     | 66.4              | 2                  | in optie   |
```

This table summarizes the key information about each house for easy review.

In [16]:
view = gr.Interface(
    fn=display_summary,
    inputs=[
        gr.Textbox(label="Landing page URL including http:// or https://")
    ],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


```markdown
| Address            | Price (€) | Living Area (m²) |
|--------------------|-----------|-------------------|
| Radarweg 297D      | 1,588     | 68.2              |
| Radarweg 297G      | 1,602     | 66.4              |
| Radarweg 297C      | 1,589     | 65.2              |
| Radarweg 297B      | 1,589     | 65.2              |
| Radarweg 297A      | 1,636     | 68.2              |
| Radarweg 297F      | 1,471     | 65.4              |
| Radarweg 297E      | 1,556     | 66.4              |
```